### Scrape the SportsReference data

In [2]:
# import packages
import urllib3
import csv
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime

#settings
http = urllib3.PoolManager()
yrs_list = ['2014', '2015', '2016', '2017', '2018']
today = datetime.datetime.today()

#define some helper functions
def monthToNum(shortMonth):
    return{
            'Jan' : 1, 'Feb' : 2, 'Mar' : 3, 'Apr' : 4, 'May' : 5, 'Jun' : 6,
            'Jul' : 7, 'Aug' : 8, 'Sep' : 9, 'Oct' : 10, 'Nov' : 11, 'Dec' : 12
    }[shortMonth]

def dateToTimeStamp(datestring, timestring):
    """
    Given the following formats for datestring and timestring, function returns a valid datetime
    datestring = 'May 28, 2005'
    timestring = '2:00 PM'
    """
    dlength = len(datestring)
    mon = monthToNum(datestring[0:3])
    year = int(datestring[dlength-4:])
    middle = datestring[3:-4]
    day = int(middle.strip().strip(','))
    if timestring == '0':
        return datetime.datetime(year, mon, day)
    else:
        tlength = len(timestring)
        M = timestring[tlength-2:]
        hour = int(timestring[:timestring.find(':')])
        if (M == 'PM') & (hour != 12):
            hour = int(timestring[:timestring.find(':')]) + 12
        minute = int(timestring[timestring.find(':')+1:timestring.find(':')+3])
    return datetime.datetime(year, mon, day, hour, minute)

def winner_home(string):
    if string == '@':
        return 0
    else:
        return 1

def concat_mult_ref_tables(filename, yrs):
    """Return a dataframe that concatenates all 
    files across a list of years, with the year set as a key
    """
    # create a list to store the dfs
    df_list = []
    headerlist = ['Year', 'Weeknum', 'Date', 'Time', 'Day', 'Team', 'Team_Pts', 'At_sym', 'Opp', 'Opp_Pts', 'Notes']
    
    for yr in yrs: 
        temp_df = None   #clear out the df
        temp_df = pd.read_csv('./SRinput/' + str(yr) + '/schedule' + str(yr) + '.csv', header=None)  #read in the file
        df_list.append(temp_df)
        
    final_df = pd.concat(df_list, ignore_index = True)
    final_df.columns = headerlist
    
    return final_df

#scrape the data and write the .csv files
for year in range(2014, 2019):
    with open ('./SRinput/' + str(year) + '/schedule' + str(year) + '.csv','w') as csvfile:
        wrtr = csv.writer(csvfile, delimiter=',', quotechar='"')
        url = "http://www.sports-reference.com/cfb/years/"+str(year)+"-schedule.html"
        response = http.request('GET', url)
        soup = BeautifulSoup(response.data)
        cnt = 0
        for row in soup.findAll('tr'):
            try:
                col1=row.findAll('th')
                Rank=col1[0].string
                col=row.findAll('td')
                Weeknum = col[0].get_text()
                Date = col[1].get_text()
                Time = col[2].get_text()
                Day = col[3].get_text()
                Winner = col[4].get_text()
                Pts = col[5].get_text()
                At_sym = col[6].get_text()
                Loser = col[7].get_text()
                Pts2 = col[8].get_text()
                TV = col[9].get_text()
                Notes = col[10].get_text()
                Year = year
                record = (Year, Weeknum, Date, Time, Day, Winner, Pts, At_sym, Loser, Pts2, Notes)
                wrtr.writerow(record)
                cnt += 1
                csvfile.flush()
            except:
                pass
        print("Finished writing " + str(year) + " schedule with " + str(cnt) + " rows")

/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Finished writing 2014 schedule with 868 rows


/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Finished writing 2015 schedule with 871 rows


/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Finished writing 2016 schedule with 873 rows


/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Finished writing 2017 schedule with 876 rows


/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Finished writing 2018 schedule with 832 rows


### import the data, concatenate it to create the current schedule_mstr dataframe

In [3]:
def get_nz_rank(team):
    ff = team[0:4]
    start_paren = ff.find('(')
    end_paren = ff.find(')')
    if end_paren > 0:
        return 1 - (int(team[start_paren+1:end_paren]) / 25)
    else:
        return 0
    
def drop_rank(team):
    ff = team[0:4]
    end_paren = ff.find(')')
    if end_paren > 0:
        return team[end_paren+1:].strip()  # the first character of ranked teams has a shitty unicode character
    else:
        return team  
    
def flipper(flag):
    if flag == 0:
        return 1
    if flag == 1:
        return 0

In [4]:
schedule_mstr = concat_mult_ref_tables('schedule', yrs_list)
# Fill any missing Time values with 0
values = {'Time': '0'}
schedule_mstr = schedule_mstr.fillna(value = values)
# Adjust the date column to a true datetime dtype and remove Time column
schedule_mstr['Date'] = schedule_mstr.apply(lambda x: dateToTimeStamp(x['Date'], x['Time']), axis=1)
schedule_mstr = schedule_mstr[schedule_mstr['Date'] < today]
schedule_mstr = schedule_mstr.drop('Time', axis=1)
# Extract rankings where applicable from team names
schedule_mstr['Team_rank'] = schedule_mstr.apply(lambda x: get_nz_rank(x['Team']), axis=1)
schedule_mstr['Opp_rank'] = schedule_mstr.apply(lambda x: get_nz_rank(x['Opp']), axis=1)
# Create binary value for home games and drop @ symbol column
schedule_mstr['Game_home'] = schedule_mstr.apply(lambda x: winner_home(x['At_sym']), axis=1)
schedule_mstr = schedule_mstr.drop('At_sym', axis=1)
# Clean up team names
schedule_mstr['Team'] = schedule_mstr.apply(lambda x: drop_rank(x['Team']), axis=1)
schedule_mstr['Opp'] = schedule_mstr.apply(lambda x: drop_rank(x['Opp']), axis=1)
# Drop any cancelled games (games with NaN in the scores column)
canc_games_list = schedule_mstr[schedule_mstr['Team_Pts'].isnull()]
schedule_mstr = schedule_mstr.drop(canc_games_list.index.values.astype(int))
# Add won column before adding loser rows
schedule_mstr['Won'] = 1

### Extend the schedule_mstr dataframe to become a full dataset with all team game results, by team

In [5]:
dopplegngr = schedule_mstr.copy()
# rename columns to perform the 'swap'
dopplegngr.columns = ['Year', 'Weeknum', 'Date', 'Day', 'Opp', 'Opp_Pts', 'Team',
                      'Team_Pts', 'Notes', 'Opp_rank', 'Team_rank', 'Game_home', 'Won']
# rearrange the columns so the axis matches the original
cols = ['Year', 'Weeknum', 'Date', 'Day', 'Team', 'Team_Pts', 
                        'Opp', 'Opp_Pts', 'Notes', 'Team_rank', 'Opp_rank', 'Game_home', 'Won']
dopplegngr = dopplegngr[cols]
dopplegngr['Game_home'] = dopplegngr.apply(lambda x: flipper(x['Game_home']), axis=1)
dopplegngr['Won'] = dopplegngr.apply(lambda x: flipper(x['Won']), axis=1)

schedule_mstr = pd.concat([schedule_mstr, dopplegngr])
schedule_mstr = schedule_mstr.sort_index()

In [6]:
schedule_mstr

,Year,Weeknum,Date,Day,Team,Team_Pts,Opp,Opp_Pts,Notes,Team_rank,Opp_rank,Game_home,Won
0,2014,1,2014-08-27 19:00:00,Wed,Georgia State,38.0,Abilene Christian,37.0,NaN,0.00,0.00,1,1
0,2014,1,2014-08-27 19:00:00,Wed,Abilene Christian,37.0,Georgia State,38.0,NaN,0.00,0.00,0,0
1,2014,1,2014-08-28 19:00:00,Thu,Howard,0.0,Akron,41.0,NaN,0.00,0.00,0,0
1,2014,1,2014-08-28 19:00:00,Thu,Akron,41.0,Howard,0.0,NaN,0.00,0.00,1,1
2,2014,1,2014-08-28 22:30:00,Thu,Arizona State,45.0,Weber State,14.0,NaN,0.24,0.00,1,1
2,2014,1,2014-08-28 22:30:00,Thu,Weber State,14.0,Arizona State,45.0,NaN,0.00,0.24,0,0
3,2014,1,2014-08-28 19:00:00,Thu,Central Michigan,20.0,Chattanooga,16.0,NaN,0.00,0.00,1,1
3,2014,1,2014-08-28 19:00:00,Thu,Chattanooga,16.0,Central Michigan,20.0,NaN,0.00,0.00,0,0
4,2014,1,2014-08-28 19:02:00,Thu,Louisiana-Monroe,17.0,Wake Forest,10.0,NaN,0.00,0.00,1,1
4,2014,1,2014-08-28 19:02:00,Thu,Wake Forest,10.0,Louisiana-Monroe,17.0,NaN,0.00,0.00,0,0


In [7]:
gametime_mstr = schedule_mstr.copy()
cols = ['Year', 'Date', 'Team', 'Opp', 'Won', 'Game_home', 'Team_rank', 'Opp_rank']
gametime_mstr = gametime_mstr[cols]
gametime_mstr.set_index('Year', inplace=True)
gametime_mstr['Season'] = gametime_mstr.index
cols1 = ['Season', 'Date', 'Team', 'Opp', 'Won', 'Game_home', 'Team_rank', 'Opp_rank']
gametime_mstr = gametime_mstr[cols1]

In [8]:
first_year = 2014

def get_season_str_yr(gamedate):
    """ Takes the date of a game and 
    returns the season year as a string"""
    if gamedate.month == 1:  # if this is a bowl game
        str_year = str(gamedate.year - 1)
    else:
        str_year = str(gamedate.year)
    return str_year

def get_season_yr(gamedate):
    """ Takes the date of a game and 
    returns the season year as an int"""
    if gamedate.month == 1:  # if this is a bowl game
        year = gamedate.year - 1
    else:
        year = gamedate.year
    return year

def season_record_to_date(team, date):
    """Given a team and date, this function returns the season win percentage as a float
    up to, but not including, that date.  If this is the first game of the season
    it returns the percentage from last season"""
    # account for bowl games that occur in next calendar year
    str_year = get_season_yr(date)
    # locate the full season for this team and calculate wins
    team_season = gametime_mstr[gametime_mstr['Team'] == team].loc[str_year]
    if isinstance(team_season, pd.core.series.Series):
        games = 0
    else:
        games_to_date = team_season[team_season['Date'] < date]
        games = games_to_date.shape[0]
    # account for first game of the season - use last year unless we don't have it
    if ((games == 0) & (str_year != first_year)):
        str_year = date.year - 1
        # Handle errors when there is no last season
        try:
            last_season = gametime_mstr[gametime_mstr['Team'] == team].loc[str_year]
            games = last_season.shape[0]
            wins = last_season['Won'].sum()
            # don't allow to divide by zero
            if games > 0:
                win_perc = wins / games
            else:
                win_perc = 0
        except KeyError:
            win_perc = 0
    elif ((games == 0) & (str_year == first_year)):
        return 0
    else:
        wins = games_to_date['Won'].sum()
        win_perc = wins / games
    
    return round(win_perc, 3)

def conf_record_to_date(team, date):
    """Given a team and date, this function returns the season win percentage against 
    conference teams as a float up to, but not including, that date.  If this is the first 
    conference game of the season it returns the win percentage from last season"""
    # account for bowl games that occur in next calendar year
    str_year = get_season_yr(date)
    # locate the full season for this team and calculate wins
    team_season = gametime_mstr[gametime_mstr['Team'] == team].loc[str_year][gametime_mstr[gametime_mstr['Team'] == team].loc[str_year]['Game_conf'] == 1]
    if isinstance(team_season, pd.core.frame.DataFrame):
        games_to_date = team_season[team_season['Date'] < date]
        games = games_to_date.shape[0]
    else:
        games = 0
    # account for first game of the season - use last year unless we don't have it
    if ((games == 0) & (str_year != first_year)):
        str_year = date.year - 1
        # Handle errors when there is no last season
        try:
            last_season = gametime_mstr[gametime_mstr['Team'] == team].loc[str_year][gametime_mstr[gametime_mstr['Team'] == team].loc[str_year]['Game_conf'] == 1]
            if isinstance(last_season, pd.core.frame.DataFrame):
                games = last_season.shape[0]
                wins = last_season['Won'].sum()
            else:
                games = 0
                wins = 0
            # don't allow to divide by zero
            if games > 0:
                win_perc = wins / games
            else:
                win_perc = 0
        except KeyError:
            win_perc = 0
    elif ((games == 0) & (str_year == first_year)):
        return 0
    else:
        wins = games_to_date['Won'].sum()
        win_perc = wins / games
    
    return round(win_perc, 3)

def get_conf(team, yr):
    try:
        conf_hist = conference_mstr[conference_mstr['School'] == team]
        conf_hist = conf_hist[(conf_hist['From'] <= yr) & (conf_hist['To'] >= yr)]
        return conf_hist['Conf'].values[0]
    except IndexError:
        return 'missing'

def in_conf_game(team1, team2, yr):
    if get_conf(team1, yr) == get_conf(team2, yr):
        return 1
    else:
        return 0
    


### import any necessary files for feature engineering

In [9]:
conference_mstr = pd.read_csv('./input/conference_master.csv')

### working apply functions to generate features

In [10]:
gametime_mstr['Game_conf'] = gametime_mstr.apply(lambda x: in_conf_game(x['Team'], x['Opp'], x['Season']), axis=1)
gametime_mstr['Team_SRTD'] = gametime_mstr.apply(lambda x: season_record_to_date(x['Team'], x['Date']), axis=1)
gametime_mstr['Team_CRTD'] = gametime_mstr.apply(lambda x: conf_record_to_date(x['Team'], x['Date']), axis=1)
gametime_mstr['Opp_SRTD'] = gametime_mstr.apply(lambda x: season_record_to_date(x['Opp'], x['Date']), axis=1)
gametime_mstr['Opp_CRTD'] = gametime_mstr.apply(lambda x: conf_record_to_date(x['Opp'], x['Date']), axis=1)

### non-working...under construction

# Model Section below assumes a valid gametime_mstr df

In [11]:
gametime_mstr

,Season,Date,Team,Opp,Won,Game_home,Team_rank,Opp_rank,Game_conf,Team_SRTD,Team_CRTD,Opp_SRTD,Opp_CRTD
Year,,,,,,,,,,,,,
2014,2014,2014-08-27 19:00:00,Georgia State,Abilene Christian,1,1,0.00,0.00,0,0.000,0.000,0.000,0.000
2014,2014,2014-08-27 19:00:00,Abilene Christian,Georgia State,0,0,0.00,0.00,0,0.000,0.000,0.000,0.000
2014,2014,2014-08-28 19:00:00,Howard,Akron,0,0,0.00,0.00,0,0.000,0.000,0.000,0.000
2014,2014,2014-08-28 19:00:00,Akron,Howard,1,1,0.00,0.00,0,0.000,0.000,0.000,0.000
2014,2014,2014-08-28 22:30:00,Arizona State,Weber State,1,1,0.24,0.00,0,0.000,0.000,0.000,0.000
2014,2014,2014-08-28 22:30:00,Weber State,Arizona State,0,0,0.00,0.24,0,0.000,0.000,0.000,0.000
2014,2014,2014-08-28 19:00:00,Central Michigan,Chattanooga,1,1,0.00,0.00,0,0.000,0.000,0.000,0.000
2014,2014,2014-08-28 19:00:00,Chattanooga,Central Michigan,0,0,0.00,0.00,0,0.000,0.000,0.000,0.000
2014,2014,2014-08-28 19:02:00,Louisiana-Monroe,Wake Forest,1,1,0.00,0.00,0,0.000,0.000,0.000,0.000


In [12]:
missing_val_count_by_column = (gametime_mstr.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])
df = gametime_mstr
null_data = df[df.isnull().any(axis=1)]
null_data

Series([], dtype: int64)


,Season,Date,Team,Opp,Won,Game_home,Team_rank,Opp_rank,Game_conf,Team_SRTD,Team_CRTD,Opp_SRTD,Opp_CRTD
Year,,,,,,,,,,,,,


### Train the Decision Tree, Random Forest & XGBoost models

In [13]:
# Decision Tree, Random Forest & XGBoost packages
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

features = gametime_mstr.columns[5:]

y = gametime_mstr.Won
X = gametime_mstr[features]

train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=1)

DT_model = DecisionTreeClassifier(random_state=1)
RF_model = RandomForestClassifier(random_state=1)
XG_model = XGBClassifier()

DT_model.fit(train_X, train_y)
RF_model.fit(train_X, train_y)
XG_model.fit(train_X, train_y)

DT_preds = DT_model.predict(val_X)
RF_preds = RF_model.predict(val_X)
XG_preds = XG_model.predict(val_X)

DT_scores = accuracy_score(val_y, DT_preds)
RF_scores = accuracy_score(val_y, RF_preds)
XG_scores = accuracy_score(val_y, XG_preds)

# Print the scores for each model
print('DT - Accuracy: ' + str(DT_scores) + '<br>')
print('RF - Accuracy: ' + str(RF_scores) + '<br>')
print('XG - Accuracy: ' + str(XG_scores) + '<br><br>')

/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


DT - Accuracy: 0.6453433678269049<br>
RF - Accuracy: 0.6787394167450611<br>
XG - Accuracy: 0.7253057384760113<br><br>


### Decision Trees, Random Forest and XGBoost model results
BASELINE(Home, Rank, SeasonRecord)<br>
DT - Accuracy: 0.6726246472248354<br>
RF - Accuracy: 0.6834430856067732<br>
XG - Accuracy: 0.7126058325493885<br><br>
Added Game_conf and Conf Records for each team<br>
DT - Accuracy: 0.6453433678269049<br>
RF - Accuracy: 0.6787394167450611<br>
XG - Accuracy: 0.7253057384760113<br><br>

In [46]:
XG_model_final = XGBClassifier()
XG_model_final.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### Save the latest XGBoost model to disk for later predictions

In [50]:
import joblib
#save model
filename = './SRmodels/finalized_XGmodel.sav'
joblib.dump(XG_model_final, filename) 

['./SRmodels/finalized_XGmodel.sav']

### Keras classifer neural network model training

In [18]:
import numpy as np
import time

# TensorFlow packages
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
#from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
#from sklearn.pipeline import Pipeline

# settings
start_time=time.time()

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# specify prediction variable and features
features = gametime_mstr.columns[5:]
y = gametime_mstr.Won
X = gametime_mstr[features]

# save input dimensions for first layer of model
X_dim = X.shape[1]

# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(128, input_dim=X_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

modelsave = create_baseline()

# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=50, batch_size=100)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, y, cv=kfold)
print("50 epochs, 100 batchsize, 5 splits")
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

end_time=time.time()
print("total time: ",end_time-start_time)

Epoch 1/50
6800/6800 [==============================] - 3s 380us/step - loss: 0.6468 - acc: 0.6874
Epoch 2/50
6800/6800 [==============================] - 1s 76us/step - loss: 0.5688 - acc: 0.7132
Epoch 3/50
6800/6800 [==============================] - 0s 64us/step - loss: 0.5564 - acc: 0.7184
Epoch 4/50
6800/6800 [==============================] - 0s 59us/step - loss: 0.5527 - acc: 0.7190
Epoch 5/50
6800/6800 [==============================] - 1s 146us/step - loss: 0.5503 - acc: 0.7216
Epoch 6/50
6800/6800 [==============================] - 1s 101us/step - loss: 0.5483 - acc: 0.7269
Epoch 7/50
6800/6800 [==============================] - 1s 123us/step - loss: 0.5453 - acc: 0.7249
Epoch 8/50
6800/6800 [==============================] - 1s 189us/step - loss: 0.5444 - acc: 0.7268
Epoch 9/50
6800/6800 [==============================] - 1s 199us/step - loss: 0.5427 - acc: 0.7324
Epoch 10/50
6800/6800 [==============================] - 2s 261us/step - loss: 0.5419 - acc: 0.7307
Epoch 11/50


6802/6802 [==============================] - 1s 100us/step - loss: 0.5374 - acc: 0.7311
Epoch 33/50
6802/6802 [==============================] - 0s 66us/step - loss: 0.5368 - acc: 0.7304
Epoch 34/50
6802/6802 [==============================] - 0s 63us/step - loss: 0.5384 - acc: 0.7286
Epoch 35/50
6802/6802 [==============================] - 0s 54us/step - loss: 0.5374 - acc: 0.7313
Epoch 36/50
6802/6802 [==============================] - 0s 59us/step - loss: 0.5366 - acc: 0.7299
Epoch 37/50
6802/6802 [==============================] - 1s 95us/step - loss: 0.5363 - acc: 0.7314
Epoch 38/50
6802/6802 [==============================] - 1s 106us/step - loss: 0.5370 - acc: 0.7298
Epoch 39/50
6802/6802 [==============================] - 1s 110us/step - loss: 0.5382 - acc: 0.7264
Epoch 40/50
6802/6802 [==============================] - 1s 75us/step - loss: 0.5362 - acc: 0.7304
Epoch 41/50
6802/6802 [==============================] - 1s 104us/step - loss: 0.5360 - acc: 0.7305
Epoch 42/50
6802/6

6802/6802 [==============================] - 1s 76us/step - loss: 0.5393 - acc: 0.7314
Epoch 14/50
6802/6802 [==============================] - 0s 60us/step - loss: 0.5376 - acc: 0.7367
Epoch 15/50
6802/6802 [==============================] - 0s 60us/step - loss: 0.5383 - acc: 0.7314
Epoch 16/50
6802/6802 [==============================] - 0s 64us/step - loss: 0.5365 - acc: 0.7354
Epoch 17/50
6802/6802 [==============================] - 1s 77us/step - loss: 0.5364 - acc: 0.7346
Epoch 18/50
6802/6802 [==============================] - 0s 62us/step - loss: 0.5363 - acc: 0.7338
Epoch 19/50
6802/6802 [==============================] - 1s 84us/step - loss: 0.5377 - acc: 0.7304
Epoch 20/50
6802/6802 [==============================] - 0s 70us/step - loss: 0.5372 - acc: 0.7329
Epoch 21/50
6802/6802 [==============================] - 1s 77us/step - loss: 0.5367 - acc: 0.7315
Epoch 22/50
6802/6802 [==============================] - 1s 77us/step - loss: 0.5365 - acc: 0.7324
Epoch 23/50
6802/6802 

6802/6802 [==============================] - 1s 85us/step - loss: 0.5378 - acc: 0.7307
Epoch 46/50
6802/6802 [==============================] - 0s 70us/step - loss: 0.5386 - acc: 0.7320
Epoch 47/50
6802/6802 [==============================] - 0s 56us/step - loss: 0.5379 - acc: 0.7289
Epoch 48/50
6802/6802 [==============================] - 0s 62us/step - loss: 0.5386 - acc: 0.7323
Epoch 49/50
6802/6802 [==============================] - 1s 98us/step - loss: 0.5378 - acc: 0.7314
Epoch 50/50
1700/1700 [==============================] - 1s 439us/step
50 epochs, 100 batchsize, 5 splits
Results: 72.88% (0.48%)
total time:  159.78049898147583


### KerasClassifer model results
BASELINE (Home, Rank, SeasonRecord) --
50 epochs, 100 batchsize, 5 splits
Results: 72.16% (0.53%)<br>
Added Game_conf and Conf Records for each team --
50 epochs, 100 batchsize, 5 splits
Results: 72.89% (0.62%)<br>

In [45]:
# keras library import  for Saving and loading model and weights
from tensorflow.python.keras.models import load_model

# save model to HDF5
modelsave.save("./SRmodels/Keras_model.h5")

In [44]:
type(estimator)

tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier